# Evaluate RAG System

In [8]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from groq import Client
from rag_be.data_loader import extract_text_from_pdf, chunk_text, chunk_per_row, retrieve_chunks
import os

# Load data
pdf_text = extract_text_from_pdf("../rag_be/doc/makanan-sehat.pdf")
pdf_chunks = chunk_text(pdf_text)

df = pd.read_csv("../rag_be/doc/fast-food.csv")
csv_chunks = chunk_per_row(df)

chunks = pdf_chunks + csv_chunks

# Load FAISS Index
vector_store_path = "../rag_be/doc/vector_store.index"
model = SentenceTransformer('all-MiniLM-L6-v2')

if os.path.exists(vector_store_path):
    index = faiss.read_index(vector_store_path)
else:
    chunk_embeddings = model.encode(chunks)
    faiss.normalize_L2(chunk_embeddings)

    dimension = chunk_embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(chunk_embeddings))
    faiss.write_index(index, vector_store_path)

# Initialize LLM API
client = Client(api_key="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")

In [21]:
questions = [
    "Apa yang dimaksud dengan fast food?",
    "Berapa kalori yang dimiliki oleh egg mcmuffin mcdollops?",
    "Apa saja jenis superfood?",
    "Bagaimana perbandingan realfood dan makanan instan?",
    "Apa jenis makanan yang cocok untuk olahraga?",
    "Bagaimana cara mengatasi emotional eating?",
    "Apa itu diet mediterania?",
    "Gejala apa yang muncul karena alergi makanan?",
    "Apa tujuan zat aditif pada makanan?",
    "Apa standar label makanan di Indonesia?",
    "Apa makanan dari raja burger yang paling tinggi kalori?",
    "Berapa total lemak tak jenuh yang dimiliki Blue Raspberry Freeze?"
]

In [22]:
# Generate answer using Groq API
def generate_answer(client, query, context):
    prompt = f"Gunakan informasi ini untuk menjawab pertanyaa, apabila jawaban tidak ada maka katakan saya tidak tahu.\n\nContext: {context}\n\nQuestion: {query}\nAnswer:"
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# RAG pipeline
def rag_answer(client,model, index, chunks, query):
    context_chunks = retrieve_chunks(model, index, chunks, query)
    context = " ".join(context_chunks)
    return generate_answer(client, query, context)

In [23]:
rag_answers = []
for q in questions:
    answer = rag_answer(client, model, index, chunks, q)
    rag_answers.append(answer)

rag_df = pd.DataFrame({"Question": questions, "RAG_Answer": rag_answers})

In [24]:
rag_df

,Question,RAG_Answer
0,Apa yang dimaksud dengan fast food?,Makanan cepat saji.
1,Berapa kalori yang dimiliki oleh egg mcmuffin ...,300.0
2,Apa saja jenis superfood?,Berikut adalah beberapa jenis superfood yang d...
3,Bagaimana perbandingan realfood dan makanan in...,Berikut adalah perbandingan antara real food d...
4,Apa jenis makanan yang cocok untuk olahraga?,"Menurut konteks yang diberikan, jenis makanan ..."
5,Bagaimana cara mengatasi emotional eating?,Mengatasi emotional eating memerlukan strategi...
6,Apa itu diet mediterania?,Diet Mediterania adalah pola makan yang berfok...
7,Gejala apa yang muncul karena alergi makanan?,Gejala alergi makanan antara lain: Ruam kulit ...
8,Apa tujuan zat aditif pada makanan?,Zat aditif pada makanan memiliki beberapa tuju...
9,Apa standar label makanan di Indonesia?,Saya tidak tahu tentang standar label makanan ...


In [ ]:
def direct_fed_doc_to_llm(client, query, doc):
    prompt = f"Gunakan informasi ini untuk menjawab pertanyaa, apabila jawaban tidak ada maka katakan saya tidak tahu.\n\nDoc: {doc}\n\nQuestion: {query}\nAnswer:"
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content 

In [26]:
doc = extract_text_from_pdf("../rag_be/doc/Summarize-PDF.pdf")
direct_answer = []
for q in questions:
    answer = direct_fed_doc_to_llm(client, q, doc)
    direct_answer.append(answer)
direct_df = pd.DataFrame({"Question": questions, "Answer": direct_answer})

In [27]:
direct_df

,Question,Answer
0,Apa yang dimaksud dengan fast food?,Fast food adalah makanan yang kaya akan lemak ...
1,Berapa kalori yang dimiliki oleh egg mcmuffin ...,Saya tidak tahu jumlah kalori yang dimiliki ol...
2,Apa saja jenis superfood?,Berikut adalah beberapa jenis superfood yang d...
3,Bagaimana perbandingan realfood dan makanan in...,Dasar makanan instan menilai tinggi gizi dan m...
4,Apa jenis makanan yang cocok untuk olahraga?,"Pada bagian ""Eksersi & Nutrisi Sinergi"", infor..."
5,Bagaimana cara mengatasi emotional eating?,Cara mengatasi emotional eating adalah dengan ...
6,Apa itu diet mediterania?,Diet mediterania adalah suatu pendekatan makan...
7,Gejala apa yang muncul karena alergi makanan?,Saya tidak tahu.
8,Apa tujuan zat aditif pada makanan?,Saya tidak tahu.
9,Apa standar label makanan di Indonesia?,Saya tidak tahu. Informasi yang diberikan tida...


In [ ]:
from rouge import Rouge

merged_df = pd.merge(rag_df, direct_df, on="Question", how="inner", suffixes=("_rag", "_direct"))

model = SentenceTransformer("all-MiniLM-L6-v2")

rouge = Rouge()

def compute_metrics(row):
    answer_rag = row["RAG_Answer"]
    answer_direct = row["Answer"]
    
    emb_rag = model.encode(answer_rag, convert_to_tensor=True)
    emb_direct = model.encode(answer_direct, convert_to_tensor=True)
    
    # Cosine similarity
    similarity = util.pytorch_cos_sim(emb_rag, emb_direct).item()
    
    # ROUGE Score
    scores = rouge.get_scores(answer_rag, answer_direct, avg=True)
    
    return pd.Series({
        "Cosine_Similarity": similarity,
        "ROUGE-1": scores["rouge-1"]["f"],
        "ROUGE-2": scores["rouge-2"]["f"],
        "ROUGE-L": scores["rouge-l"]["f"]
    })

metrics_df = merged_df.apply(compute_metrics, axis=1)

result_df = pd.concat([merged_df, metrics_df], axis=1)

print(result_df[["Question", "Cosine_Similarity", "ROUGE-1", "ROUGE-2", "ROUGE-L"]])

result_df.to_csv("comparison_results.csv", index=False)

                                             Question  Cosine_Similarity  \
0                 Apa yang dimaksud dengan fast food?           0.460246   
1   Berapa kalori yang dimiliki oleh egg mcmuffin ...           0.053627   
2                           Apa saja jenis superfood?           0.883153   
3   Bagaimana perbandingan realfood dan makanan in...           0.782311   
4        Apa jenis makanan yang cocok untuk olahraga?           0.699645   
5          Bagaimana cara mengatasi emotional eating?           0.691518   
6                           Apa itu diet mediterania?           0.802409   
7       Gejala apa yang muncul karena alergi makanan?           0.453183   
8                 Apa tujuan zat aditif pada makanan?           0.367969   
9             Apa standar label makanan di Indonesia?           0.963467   
10  Apa makanan dari raja burger yang paling tingg...           0.734903   
11  Berapa total lemak tak jenuh yang dimiliki Blu...           0.921324   

     ROUGE-

# Summary
Some of the answer for similarity like "Apa standar label makanan di Indonesia?" got 0.96 score, means that RAG generated answer and direct answer are almost same, but at some case like "Berapa kalori yang dimiliki oleh egg mcmuffin mcdollops?" have very low similarity, this may happen because the limited information inside the summarized document fed directly to the llm due to token limitation for the model.

Zero ROUGE Scores: Several questions (e.g., "Apa yang dimaksud dengan fast food?", "Berapa kalori yang dimiliki oleh egg mcmuffin?", "Gejala apa yang muncul karena alergi makanan?") have 0 ROUGE scores. This suggests that the RAG-generated answer and direct answer have no overlapping words, which means they might be completely different or the RAG model failed to retrieve relevant information.

High ROUGE Scores:
"Apa standar label makanan di Indonesia?" has the highest ROUGE-1 (0.818) and ROUGE-2 (0.667), meaning the textual overlap is very strong.
"Apa saja jenis superfood?" (ROUGE-1 = 0.48) also has decent word overlap.

# Room of Improvement
- Improve retreival quality, i.e query reformulation to get better search result
- Use more complex LLM, test using larger model
- Do more experiment about chunking and embedding model, experiment with different chunk size and other embedding model